In [34]:
import pandas as pd,numpy as np, seaborn as sns, matplotlib.pyplot as plt, joblib
from imblearn.over_sampling import RandomOverSampler
from scipy import stats
from sklearn.impute import KNNImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

In [2]:
df = pd.read_csv('../0. Dataset/Training set after splitting.csv')
df.head()

,CustomerID,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,DroppedCalls,...,IncomeGroup,OwnsMotorcycle,AdjustmentsToCreditRating,HandsetPrice,MadeCallToRetentionTeam,CreditRating,PrizmCode,Occupation,MaritalStatus,Churn
0,3317770,39.79,234.0,55.0,0.25,0.0,1.7,31.0,8.8,6.7,...,0,No,0,Unknown,No,2-High,Suburban,Other,Unknown,No
1,3224894,60.19,551.0,60.0,0.74,0.0,0.3,-88.0,-10.2,2.7,...,7,No,0,150,No,2-High,Suburban,Crafts,Yes,No
2,3231266,85.41,642.0,28.0,0.25,134.0,0.0,129.0,22.8,29.7,...,4,No,0,Unknown,No,2-High,Suburban,Crafts,No,No
3,3184922,30.18,281.0,30.0,0.00,0.0,0.0,132.0,-0.2,4.0,...,3,No,0,150,No,3-Good,Suburban,Other,Unknown,Yes
4,3350054,179.29,1459.0,95.0,0.50,118.0,48.6,-1360.0,-77.0,21.7,...,4,No,0,Unknown,No,3-Good,Rural,Other,Unknown,Yes


In [3]:
df.duplicated().sum()

0

In [4]:
df.columns

Index(['CustomerID', 'MonthlyRevenue', 'MonthlyMinutes',
       'TotalRecurringCharge', 'DirectorAssistedCalls', 'OverageMinutes',
       'RoamingCalls', 'PercChangeMinutes', 'PercChangeRevenues',
       'DroppedCalls', 'BlockedCalls', 'UnansweredCalls', 'CustomerCareCalls',
       'ThreewayCalls', 'ReceivedCalls', 'OutboundCalls', 'InboundCalls',
       'PeakCallsInOut', 'OffPeakCallsInOut', 'DroppedBlockedCalls',
       'CallForwardingCalls', 'CallWaitingCalls', 'MonthsInService',
       'UniqueSubs', 'ActiveSubs', 'ServiceArea', 'Handsets', 'HandsetModels',
       'CurrentEquipmentDays', 'AgeHH1', 'AgeHH2', 'ChildrenInHH',
       'HandsetRefurbished', 'HandsetWebCapable', 'TruckOwner', 'RVOwner',
       'Homeownership', 'BuysViaMailOrder', 'RespondsToMailOffers',
       'OptOutMailings', 'NonUSTravel', 'OwnsComputer', 'HasCreditCard',
       'RetentionCalls', 'RetentionOffersAccepted', 'NewCellphoneUser',
       'NotNewCellphoneUser', 'ReferralsMadeBySubscriber', 'IncomeGroup',
    

In [5]:
df.dtypes

CustomerID                     int64
MonthlyRevenue               float64
MonthlyMinutes               float64
TotalRecurringCharge         float64
DirectorAssistedCalls        float64
OverageMinutes               float64
RoamingCalls                 float64
PercChangeMinutes            float64
PercChangeRevenues           float64
DroppedCalls                 float64
BlockedCalls                 float64
UnansweredCalls              float64
CustomerCareCalls            float64
ThreewayCalls                float64
ReceivedCalls                float64
OutboundCalls                float64
InboundCalls                 float64
PeakCallsInOut               float64
OffPeakCallsInOut            float64
DroppedBlockedCalls          float64
CallForwardingCalls          float64
CallWaitingCalls             float64
MonthsInService                int64
UniqueSubs                     int64
ActiveSubs                     int64
ServiceArea                   object
Handsets                     float64
H

In [8]:
df.isna().sum()

CustomerID                     0
MonthlyRevenue               120
MonthlyMinutes               120
TotalRecurringCharge         120
DirectorAssistedCalls        120
OverageMinutes               120
RoamingCalls                 120
PercChangeMinutes            290
PercChangeRevenues           290
DroppedCalls                   0
BlockedCalls                   0
UnansweredCalls                0
CustomerCareCalls              0
ThreewayCalls                  0
ReceivedCalls                  0
OutboundCalls                  0
InboundCalls                   0
PeakCallsInOut                 0
OffPeakCallsInOut              0
DroppedBlockedCalls            0
CallForwardingCalls            0
CallWaitingCalls               0
MonthsInService                0
UniqueSubs                     0
ActiveSubs                     0
ServiceArea                   21
Handsets                       1
HandsetModels                  1
CurrentEquipmentDays           1
AgeHH1                       716
AgeHH2    

In [6]:
state_map = {
        'NYC': 'New York', 'LAX': 'California', 'CHI': 'Illinois', 'HAR': 'Connecticut',
        'DAL': 'Texas', 'SAN': 'Texas', 'MIA': 'Florida', 'MIL': 'Wisconsin', 'NCR': 'North Carolina',
        'PHI': 'Pennsylvania', 'OKC': 'Oklahoma', 'MIN': 'Minnesota', 'OHI': 'Ohio', 'IND': 'Indiana',
        'SFR': 'California', 'LOU': 'Kentucky', 'APC': 'Maryland', 'BOS': 'Massachusetts', 'NOL': 'Louisiana',
        'SEA': 'Washington', 'DET': 'Michigan', 'PHX': 'Arizona', 'NEV': 'Nevada', 'OMA': 'Nebraska',
        'NNY': 'New York', 'STL': 'Missouri', 'FLN': 'Florida', 'HOU': 'Texas', 'PIT': 'Pennsylvania',
        'KCY': 'Kansas', 'ATL': 'Georgia', 'AIR': 'South Carolina', 'NMX': 'New Mexico', 'SHE': 'Virginia',
        'LAU': 'Mississippi', 'SLC': 'Utah', 'HWI': 'Hawaii', 'NSH': 'Tennessee', 'BIR': 'Alabama',
        'AWI': 'Wisconsin', 'DEN': 'Colorado', 'NOR': 'Minnesota', 'GCW': 'Mississippi', 'NVU': 'Nevada',
        'LAW': 'Texas', 'VAH': 'Virginia', 'SDA': 'South Dakota', 'SEW': 'Oregon', 'HOP': 'Montana',
        'SFU': 'California'
    }

def get_state(code):
    if not isinstance(code, str) or len(code) < 3:
        return np.nan
    metro_code = code[:3] 
    if metro_code == 'SAN':
        area_code = code[-3:] if len(code) >= 9 else ''
        if area_code in ['512', '210', '830', '254', '361', '956']:  # Texas area codes
            return 'Texas'
        return 'California'  # Default for 'SAN' in other contexts 
    return state_map.get(metro_code, np.nan)

def selectBestFeatures(df, target):
    selected_features = []
    for column in df.columns:
        if column != target:
            corr,pval = stats.pearsonr(df[column],df[target])
            if pval < 0.05:
                selected_features.append(column)
            #print("'{column}' has correlation of '{corr}' and pvalue of '{pval}'".format(column=column,corr=corr,pval=pval))
    return selected_features

def remove_outliers_using_IQR(df,column):
    first_quartile, third_quartile = np.percentile(df[column],25), np.percentile(df[column],75)
    IQR = third_quartile - first_quartile
    floor = first_quartile - 1.5 * IQR
    cieling = third_quartile + 1.5 * IQR
    outlier_mask = (df[column] < floor) | (df[column] > cieling)
    print()
    df = df[~outlier_mask]
    return df

In [7]:
df.drop(columns=['CustomerID'],axis=1,inplace=True)
df['HandsetPrice'] = df['HandsetPrice'].replace('Unknown', np.nan)
df['HandsetPrice'] = pd.to_numeric(df['HandsetPrice'])
df['MaritalStatus'] = df['MaritalStatus'].replace('Unknown', np.nan)

yes_no_features = ['Churn','ChildrenInHH','HandsetRefurbished','HandsetWebCapable','TruckOwner','RVOwner','BuysViaMailOrder',
     'RespondsToMailOffers','OptOutMailings','NonUSTravel','OwnsComputer','HasCreditCard','NewCellphoneUser','NotNewCellphoneUser',
     'OwnsMotorcycle','MadeCallToRetentionTeam']

keep_values = {'CreditRating': ['2-High','1-Highest','3-Good'], 'Occupation': ['Professional','Crafts'], 
               'PrizmCode': ['Suburban','Town'],'ServiceArea': ['Texas', 'New York', 'California', 'Florida',
               'Maryland', 'Illinois','Georgia', 'Ohio', 'North Carolina', 'Nevada', 'Pennsylvania','Michigan', 
               'Massachusetts' ,'Washington', 'Kansas'] }

for feature in yes_no_features:
    df[feature] = [1 if value == 'Yes' else 0 for value in df[feature]]
df['Homeownership'] = [1 if value == 'Known' else 0 for value in df['Homeownership']]
df['ServiceArea'] = df['ServiceArea'].apply(lambda code: get_state(code))
for feature, values in keep_values.items():
    df[feature] = [value if value in values else 'Other' for value in df[feature]]
df = pd.get_dummies(df,columns=['CreditRating','PrizmCode','Occupation','ServiceArea'])
df['MaritalStatus'] = [1 if status == 'Yes' else 0 if status =='No' else np.nan for status in df['MaritalStatus']]

In [11]:
df.head()

,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,DroppedCalls,BlockedCalls,...,ServiceArea_Massachusetts,ServiceArea_Michigan,ServiceArea_Nevada,ServiceArea_New York,ServiceArea_North Carolina,ServiceArea_Ohio,ServiceArea_Other,ServiceArea_Pennsylvania,ServiceArea_Texas,ServiceArea_Washington
0,39.79,234.0,55.0,0.25,0.0,1.7,31.0,8.8,6.7,0.7,...,False,False,False,False,False,False,False,False,False,False
1,60.19,551.0,60.0,0.74,0.0,0.3,-88.0,-10.2,2.7,32.7,...,False,False,False,False,False,False,False,False,False,False
2,85.41,642.0,28.0,0.25,134.0,0.0,129.0,22.8,29.7,0.0,...,False,False,False,True,False,False,False,False,False,False
3,30.18,281.0,30.0,0.00,0.0,0.0,132.0,-0.2,4.0,0.0,...,False,False,False,True,False,False,False,False,False,False
4,179.29,1459.0,95.0,0.50,118.0,48.6,-1360.0,-77.0,21.7,11.0,...,False,False,False,False,False,False,True,False,False,False


In [ ]:
imputer = KNNImputer(weights = 'distance')
imputer.fit(df.drop(columns=['Churn'],axis=1))
joblib.dump(imputer,'NullImputer.h5')

In [8]:
imputer = joblib.load('NullImputer.h5')
imputed_df = imputer.transform(df.drop(columns=['Churn'],axis=1))
imputed_df = pd.DataFrame(imputed_df, columns= df.drop(columns=['Churn']).columns)
imputed_df['Churn'] = df['Churn']

In [9]:
selected_features = selectBestFeatures(imputed_df, 'Churn')

In [10]:
len(selected_features)

51

In [15]:
selected_features

['MonthlyRevenue',
 'MonthlyMinutes',
 'TotalRecurringCharge',
 'DirectorAssistedCalls',
 'OverageMinutes',
 'RoamingCalls',
 'PercChangeMinutes',
 'PercChangeRevenues',
 'DroppedCalls',
 'UnansweredCalls',
 'CustomerCareCalls',
 'ThreewayCalls',
 'ReceivedCalls',
 'OutboundCalls',
 'InboundCalls',
 'PeakCallsInOut',
 'OffPeakCallsInOut',
 'CallWaitingCalls',
 'MonthsInService',
 'UniqueSubs',
 'ActiveSubs',
 'Handsets',
 'HandsetModels',
 'CurrentEquipmentDays',
 'AgeHH1',
 'AgeHH2',
 'HandsetRefurbished',
 'HandsetWebCapable',
 'Homeownership',
 'BuysViaMailOrder',
 'RespondsToMailOffers',
 'RetentionCalls',
 'RetentionOffersAccepted',
 'NotNewCellphoneUser',
 'IncomeGroup',
 'AdjustmentsToCreditRating',
 'HandsetPrice',
 'MadeCallToRetentionTeam',
 'CreditRating_1-Highest',
 'CreditRating_2-High',
 'CreditRating_3-Good',
 'CreditRating_Other',
 'PrizmCode_Suburban',
 'PrizmCode_Town',
 'ServiceArea_Florida',
 'ServiceArea_Kansas',
 'ServiceArea_Maryland',
 'ServiceArea_Michigan',
 '

In [24]:
labels = imputed_df['Churn']
features = imputed_df[selected_features]

In [25]:
labels.shape

(40837,)

In [26]:
features.shape

(40837, 51)

In [44]:
ros = RandomOverSampler(random_state=42)

# Apply SMOTE to oversample the minority class
features,labels = ros.fit_resample(features, labels)

x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=42, stratify = labels)

C:\Users\Nour Kamaly\anaconda3\lib\site-packages\sklearn\base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
C:\Users\Nour Kamaly\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(


In [35]:
xgb_classifier = XGBClassifier(n_estimators= 100)
xgb_classifier.fit(X_train,y_train)

prediction = xgb_classifier.predict(X_test)
print(classification_report(y_test,prediction))
print()

print("Precision:{}".format(precision_score(y_test,prediction)))
print("Recall:{}".format(recall_score(y_test,prediction)))
print("F1 Score:{}".format((f1_score(y_test,prediction))))

              precision    recall  f1-score   support

           0       0.75      0.69      0.72      2907
           1       0.72      0.77      0.74      2907

    accuracy                           0.73      5814
   macro avg       0.74      0.73      0.73      5814
weighted avg       0.74      0.73      0.73      5814


Precision:0.7169270003187759
Recall:0.7736498108015136
F1 Score:0.7442091330244871


In [38]:
joblib.dump(xgb_classifier,'xgb_classifierRandomSampler.h5')

['xgb_classifierRandomSampler.h5']

In [37]:
X_train.shape

(52322, 51)

In [130]:
imputed_df = imputed_df[selected_features]
imputed_df['Churn'] = df['Churn']
imputed_df.to_csv('Processed data.csv',index=False)

In [41]:
import pandas as pd,numpy as np, joblib, optuna
from imblearn.over_sampling import SMOTE
from sklearn.impute import KNNImputer
from sklearn.ensemble import BaggingClassifier,GradientBoostingClassifier, RandomForestClassifier,AdaBoostClassifier,ExtraTreesClassifier
from sklearn import tree
from sklearn.linear_model import SGDClassifier,RidgeClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB,GaussianNB,GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score, confusion_matrix

In [46]:
def create_metrics_table(model_name, y_train, y_train_predictions, truth, predictions, train_support, validation_support):
    precision_per_label = precision_score(truth, predictions, average=None)
    recall_per_label = recall_score(truth, predictions, average=None)
    f1_per_label = f1_score(truth, predictions, average=None)

    precision_per_label_train = precision_score(y_train, y_train_predictions, average=None)
    recall_per_label_train = recall_score(y_train, y_train_predictions, average=None)

    metrics = pd.DataFrame(columns=['Model', 'Training Precision', 'Training Recall', 'Train Support',
                                    'Validation Precision', 'Validation Recall', 'Validation F1 Score'],
                           index=range(len(precision_per_label)))

    for index in range(len(precision_per_label)):
        metrics.at[index, 'Training Precision'] = precision_per_label_train[index]
        metrics.at[index, 'Training Recall'] = recall_per_label_train[index]
        metrics.at[index, 'Train Support'] = train_support[index]
        metrics.at[index, 'Validation Support'] = validation_support[index]
        metrics.at[index, 'Validation Precision'] = precision_per_label[index]
        metrics.at[index, 'Validation Recall'] = recall_per_label[index]
        metrics.at[index, 'Validation F1 Score'] = f1_per_label[index]

    metrics['Model'] = model_name
    return metrics

In [51]:
from collections import Counter
train_support = Counter(y_train)
validation_support = Counter(y_test)

In [52]:
models = {
    "LogisticRegression" : LogisticRegression(),
    "AdaBoost" : AdaBoostClassifier(DecisionTreeClassifier(),n_estimators=10),
    "GradientBoosting"      : GradientBoostingClassifier(),
    "BaggingClassifier"  : BaggingClassifier(tree.DecisionTreeClassifier(random_state=1)),
    "RandomForest"      : RandomForestClassifier(n_estimators=3),
    "eXtremeGradientBoosting"     : XGBClassifier(objective= 'binary:logistic'),
    "KNN"                : KNeighborsClassifier(n_neighbors = 30, weights = 'distance'),
    "DecisionTree"      : DecisionTreeClassifier(),
    "ExtraTreeClassifier"       : ExtraTreesClassifier(n_estimators=3)
}
metrics = pd.DataFrame()
for name, model in models.items():
    

    print(f'Using model: {name}')
    model.fit(x_train, y_train)
    #joblib.dump(model,f'Saved Models/{name}.h5')
    y_train_predictions = model.predict(x_train)
    predictions = model.predict(x_test)
    metrics = pd.concat([metrics,create_metrics_table(name,y_train,y_train_predictions,y_test, predictions, train_support,validation_support)])

Using model: LogisticRegression


C:\Users\Nour Kamaly\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Using model: AdaBoost
Using model: GradientBoosting
Using model: BaggingClassifier
Using model: RandomForest
Using model: eXtremeGradientBoosting
Using model: KNN
Using model: DecisionTree
Using model: ExtraTreeClassifier


In [53]:
metrics

,Model,Training Precision,Training Recall,Train Support,Validation Precision,Validation Recall,Validation F1 Score,Validation Support
0,LogisticRegression,0.578778,0.558159,26161,0.592901,0.563123,0.577629,2907.0
1,LogisticRegression,0.573358,0.593785,26161,0.584016,0.613347,0.598322,2907.0
0,AdaBoost,1.0,1.0,26161,0.898363,0.717578,0.797858,2907.0
1,AdaBoost,1.0,1.0,26161,0.764891,0.918817,0.834818,2907.0
0,GradientBoosting,0.661928,0.596269,26161,0.6504,0.55934,0.601443,2907.0
1,GradientBoosting,0.632703,0.695463,26161,0.613458,0.699346,0.653593,2907.0
0,BaggingClassifier,0.993418,0.998012,26161,0.880602,0.844857,0.86236,2907.0
1,BaggingClassifier,0.998003,0.993387,26161,0.850909,0.885449,0.867835,2907.0
0,RandomForest,0.976856,0.953519,26161,0.855456,0.722738,0.783517,2907.0
1,RandomForest,0.954603,0.977409,26161,0.759976,0.877881,0.814685,2907.0


In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [55]:
model = Sequential([
    Dense(250, activation="relu", input_shape=(x_train.shape[1],)),
    Dropout(0.2),
    Dense(250, activation="relu"),
    Dropout(0.2),
    Dense(1, activation="sigmoid")
])

# Compile model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Early stopping and checkpoint
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True)

C:\Users\Nour Kamaly\anaconda3\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:


# Train model
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=32, callbacks=[early_stopping, checkpoint])


Epoch 1/100
1636/1636 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5251 - loss: 3.6455

1636/1636 ━━━━━━━━━━━━━━━━━━━━ 21s 10ms/step - accuracy: 0.5251 - loss: 3.6442 - val_accuracy: 0.5691 - val_loss: 0.6857
Epoch 2/100
1632/1636 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5513 - loss: 0.6903

1636/1636 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.5513 - loss: 0.6903 - val_accuracy: 0.5915 - val_loss: 0.6788
Epoch 3/100
1636/1636 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.5551 - loss: 0.6875 - val_accuracy: 0.5781 - val_loss: 0.6791
Epoch 4/100
1634/1636 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5602 - loss: 0.6851

1636/1636 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.5602 - loss: 0.6851 - val_accuracy: 0.5918 - val_loss: 0.6769
Epoch 5/100
1632/1636 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5671 - loss: 0.6827

1636/1636 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.5671 - loss: 0.6827 - val_accuracy: 0.5961 - val_loss: 0.6719
Epoch 6/100
1635/1636 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5659 - loss: 0.6825

...

1636/1636 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.5976 - loss: 0.6642 - val_accuracy: 0.6132 - val_loss: 0.6565
Epoch 35/100
1636/1636 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.5990 - loss: 0.6606 - val_accuracy: 0.6142 - val_loss: 0.6571
Epoch 36/100
1636/1636 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.6014 - loss: 0.6604 - val_accuracy: 0.6065 - val_loss: 0.6629
Epoch 37/100
1632/1636 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6015 - loss: 0.6596

1636/1636 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.6015 - loss: 0.6596 - val_accuracy: 0.6115 - val_loss: 0.6565
Epoch 38/100
1636/1636 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.5993 - loss: 0.6609 - val_accuracy: 0.6142 - val_loss: 0.6576
Epoch 39/100
1636/1636 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.6000 - loss: 0.6627 - val_accuracy: 0.6101 - val_loss: 0.6613
Epoch 40/100
1632/1636 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6009 - loss: 0.6587

1636/1636 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.6009 - loss: 0.6587 - val_accuracy: 0.6130 - val_loss: 0.6565
Epoch 41/100
1636/1636 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.6041 - loss: 0.6610 - val_accuracy: 0.5991 - val_loss: 0.6573
Epoch 42/100
1631/1636 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6013 - loss: 0.6589

1636/1636 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.6013 - loss: 0.6589 - val_accuracy: 0.6158 - val_loss: 0.6558
Epoch 43/100
1636/1636 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.5995 - loss: 0.6592 - val_accuracy: 0.6010 - val_loss: 0.6661
Epoch 44/100
1632/1636 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5987 - loss: 0.6596

1636/1636 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.5988 - loss: 0.6595 - val_accuracy: 0.6144 - val_loss: 0.6545
Epoch 45/100
1636/1636 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.6028 - loss: 0.6592 - val_accuracy: 0.5843 - val_loss: 0.6656
Epoch 46/100
1636/1636 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.6070 - loss: 0.6556 - val_accuracy: 0.6082 - val_loss: 0.6607
Epoch 47/100
1636/1636 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.6037 - loss: 0.6575 - val_accuracy: 0.6090 - val_loss: 0.6563
Epoch 48/100
1636/1636 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.6050 - loss: 0.6562 - val_accuracy: 0.6089 - val_loss: 0.6579
Epoch 49/100
1636/1636 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.6024 - loss: 0.6576 - val_accuracy: 0.6084 - val_loss: 0.6588
Epoch 50/100
 289/1636 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.6089 - loss: 0.6512